so now we will be start working on my project for the hackathon

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [79]:
df= pd.read_csv('mail_phishing_1.csv')
df.head()

,Subject,Text,Type
0,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\...,Fraud
1,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",Fraud
2,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,Fraud
3,from Mrs.Johnson,Goodday Dear\n\n\nI know this mail will come t...,Fraud
4,Co-Operation,FROM MR. GODWIN AKWESI\nTEL: +233 208216645\nF...,Fraud


In [80]:
print(df.shape)
df.isna().sum()

(159, 3)


Subject    2
Text       0
Type       0
dtype: int64

In [81]:
# we will clean the data of the na values
df.dropna(inplace=True)
df.isna().sum()


Subject    0
Text       0
Type       0
dtype: int64

In [82]:
import sklearn.preprocessing as pre

class_mapping = {'Fraud': 1, 'Phishing': 1, 'Commercial Spam': 0, 'False Positives ': 0}
df['Type'] = df['Type'].map(class_mapping)
df.head()
print(df)

                                               Subject  \
0           URGENT BUSINESS ASSISTANCE AND PARTNERSHIP   
1                  URGENT ASSISTANCE /RELATIONSHIP (P)   
2                                      GOOD DAY TO YOU   
3                                     from Mrs.Johnson   
4                                         Co-Operation   
..                                                 ...   
154                 These Bags Just Arrived For Spring   
155  POTUS Comes to Broadway this April! Get Ticket...   
156                       Let’s talk about Bridgerton!   
157                    MONDAY MIX: All eyes on Ukraine   
158  The DOTD is back on with 15% off a lightning-f...   

                                                  Text  Type  
0    URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\...     1  
1    Dear Friend,\n\nI am Mr. Ben Suleman a custom ...     1  
2    FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...     1  
3    Goodday Dear\n\n\nI know this mail will come t

In [83]:
df['Type'].value_counts()

Type
0    79
1    78
Name: count, dtype: int64

Text Preprocessing

In [84]:
import nltk

In [85]:
import string
from nltk.corpus import stopwords

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    
    stopw= set(stopwords.words('english'))
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopw ])



In [86]:
df['Text']=df['Text'].apply(process_text)
df['Subject']=df['Subject'].apply(process_text)
df.head()


,Subject,Text,Type
0,URGENT BUSINESS ASSISTANCE PARTNERSHIP,URGENT BUSINESS ASSISTANCE PARTNERSHIP DEAR FR...,1
1,URGENT ASSISTANCE RELATIONSHIP P,Dear Friend Mr Ben Suleman custom officer work...,1
2,GOOD DAY,ROYAL MAJESTY HRM CROWN RULER ELEME KINGDOM CH...,1
3,MrsJohnson,Goodday Dear know mail come surprise neverthel...,1
4,CoOperation,MR GODWIN AKWESI TEL 233 208216645 FAX 233 276...,1


In [87]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])

df['Text']=df['Text'].apply(stemming)
df['Subject']=df['Subject'].apply(stemming)
df.head()

,Subject,Text,Type
0,urgent business assistance partnership,urgent business assistance partnership dear fr...,1
1,urgent assistance relationship p,dear friend mr ben suleman custom officer work...,1
2,good day,royal majesty hrm crown ruler eleme kingdom ch...,1
3,mrsjohnson,goodday dear know mail come surprise neverthel...,1
4,cooperation,mr godwin akwesi tel 233 208216645 fax 233 276...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion

df['combined_text'] = df['Subject'] + ' ' + df['Text']

# Initialize CountVectorizer for each text feature
count_vectorizer = CountVectorizer()
text_vectorized = count_vectorizer.fit_transform(df['combined_text'])

# text_vectorized = count_vectorizer_new.transform(df['combined_text'])

print(text_vectorized.toarray().shape)

(157, 109437)


In [89]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(text_vectorized,df['Type'],test_size=0.20)

In [90]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(125, 109437)
(32, 109437)
(125,)
(32,)


In [19]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB,BernoulliNB 
# svm = SVC()
# svm= MultinomialNB()
svm=BernoulliNB()
svm.fit(X_train,y_train)

BernoulliNB()

In [91]:
print(X_test.shape)
print(type(X_test))

(32, 109437)
<class 'scipy.sparse._csr.csr_matrix'>


In [92]:
y_pred = svm.predict(X_test)

In [93]:
from sklearn.metrics import accuracy_score,classification_report

accuracy= 100*accuracy_score(y_test,y_pred,normalize= True)
print(accuracy)
print(classification_report(y_test,y_pred))

81.25
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        19
           1       1.00      0.54      0.70        13

    accuracy                           0.81        32
   macro avg       0.88      0.77      0.78        32
weighted avg       0.86      0.81      0.80        32



now we will train it some new data

In [65]:
df= pd.read_csv('mail_phishing_2.csv')
# print(df)
df.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr PW: bnaweb22 -----O...,0,1
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,0
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,1
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald, We are currently trading under GT...",0,0
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,0


In [66]:
df=df[['subject','body','label']]

In [67]:
print(df.shape)
df.isna().sum()

(6331, 3)


subject    106
body         0
label        0
dtype: int64

In [68]:
# we will clean the data of the na values
df.dropna(inplace=True)
df.isna().sum()

subject    0
body       0
label      0
dtype: int64

In [69]:
print(len(df['label']))
df['label'].value_counts()

6225


label
1    3293
0    2932
Name: count, dtype: int64

In [70]:
df['body']=df['body'].apply(process_text)
df['subject']=df['subject'].apply(process_text)
df.head()

,subject,body,label
0,FW June 29 BNA Inc Daily Labor Report,User ID enrondlr PW bnaweb22 Original Message ...,0
1,NGX failover plan,Hi Chris Tonight rolling new report Currently ...,0
2,Intranet Site,Rika r new Original Message Thomas Paul Sent F...,0
3,FW ENA Upstream Company information,JohnGerald currently trading GTC Spot contract...,0
4,New Master Physical,Gerald Stacy Attached worksheet new master phy...,0


In [71]:
df['body']=df['body'].apply(stemming)
df['subject']=df['subject'].apply(stemming)
df.head()

,subject,body,label
0,fw june 29 bna inc daily labor report,user id enrondlr pw bnaweb22 original message ...,0
1,ngx failover plan,hi chris tonight rolling new report currently ...,0
2,intranet site,rika r new original message thomas paul sent f...,0
3,fw ena upstream company information,johngerald currently trading gtc spot contract...,0
4,new master physical,gerald stacy attached worksheet new master phy...,0


In [72]:
df['combined_text'] = df['subject'] + ' ' + df['body']

# Initialize CountVectorizer for each text feature
count_vectorizer_new = CountVectorizer()

text_vectorized = count_vectorizer_new.fit_transform(df['combined_text'])


In [73]:

X_train,X_test,y_train,y_test = train_test_split(text_vectorized,df['label'],test_size=0.35)

In [74]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(4046, 109437)
(2179, 109437)
(4046,)
(2179,)


In [75]:
svm.fit(X_train,y_train)

BernoulliNB()

In [76]:
print(X_test.shape)
print(type(X_test))

(2179, 109437)
<class 'scipy.sparse._csr.csr_matrix'>


In [77]:
y_pred = svm.predict(X_test)

In [78]:
accuracy= 100*accuracy_score(y_test,y_pred,normalize= True)
print(accuracy)
print(classification_report(y_test,y_pred))

98.76089949518128
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1017
           1       1.00      0.98      0.99      1162

    accuracy                           0.99      2179
   macro avg       0.99      0.99      0.99      2179
weighted avg       0.99      0.99      0.99      2179



In [94]:
from joblib import dump, load

In [95]:
dump(svm,'D:\Kaam_ke_projects\Hack-o-mania-advitiya-2024_phishing_email_classifier\mail_phishing_detection_model.joblib')
dump(count_vectorizer_new, 'D:\Kaam_ke_projects\Hack-o-mania-advitiya-2024_phishing_email_classifier\count_vectorizer.joblib')

<>:1: SyntaxWarning: invalid escape sequence '\K'
<>:2: SyntaxWarning: invalid escape sequence '\K'
<>:1: SyntaxWarning: invalid escape sequence '\K'
<>:2: SyntaxWarning: invalid escape sequence '\K'
C:\Users\nitu2\AppData\Local\Temp\ipykernel_24736\3934125485.py:1: SyntaxWarning: invalid escape sequence '\K'
  dump(svm,'D:\Kaam_ke_projects\Hack-o-mania-advitiya-2024_phishing_email_classifier\mail_phishing_detection_model.joblib')
C:\Users\nitu2\AppData\Local\Temp\ipykernel_24736\3934125485.py:2: SyntaxWarning: invalid escape sequence '\K'
  dump(count_vectorizer_new, 'D:\Kaam_ke_projects\Hack-o-mania-advitiya-2024_phishing_email_classifier\count_vectorizer.joblib')


['D:\\Kaam_ke_projects\\Hack-o-mania-advitiya-2024_phishing_email_classifier\\count_vectorizer.joblib']